In [1]:
from dotenv import load_dotenv

import pandas as pd
from datetime import datetime
from xcontest.xcontest import login, flight_days

load_dotenv()

driver = login()
flight_day_list_xcontest = flight_days(driver,  'Metzingen' )

df = pd.DataFrame(flight_day_list_xcontest,columns=['isodate'])

df['isodate'] = df.apply(lambda row: datetime.strptime(row['isodate'], '%d.%m.%Y') ,axis=1)


['29.09.2024', '22.09.2024', '31.08.2024', '26.08.2024', '03.08.2024', '18.07.2024', '14.07.2024', '29.06.2024', '26.05.2024', '25.05.2024', '11.05.2024', '04.05.2024', '21.04.2024', '31.03.2024', '10.02.2024', '09.09.2023', '11.08.2023', '05.08.2023', '07.07.2023', '18.06.2023', '10.06.2023', '04.06.2023', '03.06.2023', '28.05.2023', '27.05.2023', '18.05.2023', '30.04.2023', '23.04.2023', '07.04.2023', '18.03.2023', '30.10.2022', '09.10.2022', '30.09.2022', '21.08.2022', '02.07.2022', '04.06.2022', '03.06.2022', '18.04.2022', '06.03.2022', '15.01.2022', '24.10.2021', '17.10.2021', '10.10.2021', '02.10.2021', '21.08.2021', '12.08.2021', '25.07.2021', '10.07.2021', '04.07.2021', '27.06.2021', '26.06.2021', '01.06.2021', '30.05.2021', '08.05.2021', '01.05.2021', '17.10.2020', '20.09.2020', '17.09.2020', '19.08.2020', '09.08.2020', '25.07.2020', '18.07.2020', '24.06.2020', '02.06.2020', '01.06.2020', '30.05.2020', '29.05.2020', '21.05.2020', '14.05.2020', '01.01.2020', '31.10.2019', '21.0

In [2]:
import dhvxc.dhvxc as dhv

flight_day_list = dhv.flight_days()
df_dhv = pd.DataFrame(flight_day_list,columns=['isodate'])

df_dhv['isodate'] = df_dhv.apply(lambda row: datetime.strptime(row['isodate'], '%Y-%m-%d') ,axis=1)


In [3]:
df_all= pd.concat([df, df_dhv],axis=0)

df_all =  pd.DataFrame(df_all['isodate'].unique(),columns=['isodate'])

df_all['year'] = df_all.apply(lambda row:row['isodate'].year ,axis=1)
df_all['weekday'] = df_all.apply(lambda row: row['isodate'].day_name() ,axis=1)
df_all = df_all[df_all['year'] >= 2017]
#print(df_all.head())

print(df_all.groupby(by='year').count().aggregate(['count','mean','std']))

df_pvt = pd.pivot_table(df_all, index='year', columns='weekday',aggfunc='count',fill_value=0 , margins=True)
df_pvt = df_pvt[[ ('isodate', 'Monday'), ('isodate', 'Tuesday'), ('isodate', 'Wednesday'),       
       ('isodate', 'Thursday'), ('isodate', 'Friday'),('isodate', 'Saturday'), ('isodate', 'Sunday'),
      ]]
df_pvt['Total']= df_pvt.apply(lambda d: d.sum(), axis=1)
df_pvt.head(10)


         isodate    weekday
count   8.000000   8.000000
mean   20.625000  20.625000
std     6.696214   6.696214


isodate                                                   Total
weekday  Monday Tuesday Wednesday Thursday Friday Saturday Sunday      
year                                                                   
2017          1       0         0        2      2        7     13    25
2018          2       1         0        0      8        8     14    33
2019          1       0         1        2      3        6      9    22
2020          1       1         3        3      1        6      2    17
2021          0       1         0        1      0        6      7    15
2022          1       0         0        0      2        3      5    11
2023          0       0         1        1      3        7      8    20
2024          3       2         1        1      1        7      7    22
All           9       5         6       10     20       50     65   165

In [36]:
print(df_pvt.to_markdown())

| year   |   ('isodate', 'Monday') |   ('isodate', 'Tuesday') |   ('isodate', 'Wednesday') |   ('isodate', 'Thursday') |   ('isodate', 'Friday') |   ('isodate', 'Saturday') |   ('isodate', 'Sunday') |   ('Total', '') |
|:-------|------------------------:|-------------------------:|---------------------------:|--------------------------:|------------------------:|--------------------------:|------------------------:|----------------:|
| 2017   |                       1 |                        0 |                          0 |                         2 |                       2 |                         7 |                      13 |              25 |
| 2018   |                       2 |                        1 |                          0 |                         0 |                       8 |                         8 |                      14 |              33 |
| 2019   |                       1 |                        0 |                          1 |                         2 |    

In [8]:
import numpy as np

from wetter.wetter import get_combined_data

timeframe_start,timeframe_end = 2017010100,2024123123

df = get_combined_data(timeframe_start,timeframe_end, 'Sun',13, 13)

print(df.tail(3))

          strength wind_alignment precip
meas_hour       13             13     13
meas_day                                
20240915       4.5       0.750000    0.0
20240922       2.0       0.969846    0.0
20240929       1.7       0.413176    0.0


In [11]:
from datetime import datetime

flight_day_as_mess_date = [int(datetime.strftime(d, '%Y%m%d')) for d in df_all['isodate']]

df['is_flight_day'] = df.apply(lambda row: 1 if int(row.name) in flight_day_as_mess_date else 0, axis=1)
#print(df.head(10))

X, y = df[['strength','wind_alignment','precip']].to_numpy(), df['is_flight_day'].to_numpy() # features and labels
training = len(df[df.index < '20230101'])

print(X.shape, y.shape, training)

(403, 3) (403,) 312


In [15]:
# perceptron ?  https://en.wikipedia.org/wiki/Perceptron - no separating hyperplane
from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors=7, weights= 'distance')
neigh.fit(X[:training], y[:training])

df['knn'] =  df.apply(lambda row:  neigh.predict([[row['strength',13],row['wind_alignment',13],row['precip',13]]])[0], axis=1)

df_missed = df[(df.index >= '20230101') & (df['knn']>= 0.4) & (df['is_flight_day']==0)].copy()
print(df_missed.head(10))

          strength wind_alignment precip is_flight_day       knn
meas_hour       13             13     13                        
meas_day                                                        
20230108       4.4   1.499760e-32    0.0             0  0.568460
20230122       3.3   1.169778e-01    0.0             0  0.430423
20230205       2.5   5.868241e-01    0.0             0  1.000000
20230319       3.9   1.169778e-01    0.0             0  1.000000
20230416       2.1   7.500000e-01    0.0             0  0.526428
20230806       2.9   1.169778e-01    0.0             0  0.441341
20231001       4.1   3.015369e-02    0.0             0  0.593750
20231126       1.8   1.169778e-01    0.0             0  0.555261
20240107       2.3   3.015369e-02    0.0             0  0.547060
20240211       2.0   4.131759e-01    0.3             0  0.415236


In [17]:
from sklearn import tree

rt = tree.DecisionTreeClassifier()
rt.fit(X[:training], y[:training])

df['tree'] =  df.apply(lambda row:  rt.predict([[row['strength',13],row['wind_alignment',13],row['precip',13]]])[0], axis=1)

df_missed = df[(df.index >= '20230101') & (df['tree']>= 0.5) & (df['is_flight_day']==0)].copy()
print(df_missed.head(20))

          strength wind_alignment precip is_flight_day       knn tree
meas_hour       13             13     13                             
meas_day                                                             
20230108       4.4   1.499760e-32    0.0             0  0.568460    1
20230122       3.3   1.169778e-01    0.0             0  0.430423    1
20230205       2.5   5.868241e-01    0.0             0  1.000000    1
20230319       3.9   1.169778e-01    0.0             0  1.000000    1
20230416       2.1   7.500000e-01    0.0             0  0.526428    1
20230806       2.9   1.169778e-01    0.0             0  0.441341    1
20231126       1.8   1.169778e-01    0.0             0  0.555261    1
20240519       2.9   0.000000e+00    0.0             0  1.000000    1


In [18]:
from sklearn import ensemble

rf = ensemble.RandomForestClassifier()
rf.fit(X[:training], y[:training])

df['forest'] =  df.apply(lambda row:  rf.predict([[row['strength',13],row['wind_alignment',13],row['precip',13]]])[0], axis=1)

df_missed = df[(df.index >= '20230101') & ((df['knn']>= 0.4) | (df['tree']>= 0.5) | (df['forest']>= 0.5)) & (df['is_flight_day']==0)].copy()
print(df_missed.head(20))

          strength wind_alignment precip is_flight_day       knn tree forest
meas_hour       13             13     13                                    
meas_day                                                                    
20230108       4.4   1.499760e-32    0.0             0  0.568460    1      0
20230122       3.3   1.169778e-01    0.0             0  0.430423    1      1
20230205       2.5   5.868241e-01    0.0             0  1.000000    1      1
20230319       3.9   1.169778e-01    0.0             0  1.000000    1      1
20230416       2.1   7.500000e-01    0.0             0  0.526428    1      1
20230806       2.9   1.169778e-01    0.0             0  0.441341    1      1
20230910       1.7   8.830222e-01    0.0             0  0.397727    0      1
20231001       4.1   3.015369e-02    0.0             0  0.593750    0      0
20231126       1.8   1.169778e-01    0.0             0  0.555261    1      0
20240107       2.3   3.015369e-02    0.0             0  0.547060    0      0

In [22]:
# precision, recall, f1-score
for model in ['knn','tree','forest']:
    threshold = 0.4 if model=='knn' else 0.5
    tp = len(df[(df.index >= '20230101') & (df[model]>= threshold) & (df['is_flight_day']==1)])
    fp = len(df[(df.index >= '20230101') & (df[model]>= threshold) & (df['is_flight_day']==0)])
    fn = len(df[(df.index >= '20230101') & (df[model]< threshold) & (df['is_flight_day']==1)])
    # print(tp,fp,fn)
    
    precision, recall = tp/(tp+fp), tp/(tp+fn)
    print(model, precision, recall, 2*precision*recall/(precision+recall))

knn 0.35 0.4666666666666667 0.4
tree 0.4666666666666667 0.4666666666666667 0.4666666666666667
forest 0.46153846153846156 0.4 0.42857142857142855


In [23]:
df_actual = df[(df['is_flight_day']==1)].copy()

df_actual.tail(10)

,strength,wind_alignment,precip,is_flight_day,knn,tree,forest
meas_hour,13,13,13,,,,
meas_day,,,,,,,
20230625,2.4,1.499760e-32,0.0,1,4.127602e-01,1,1
20230924,3.1,1.169778e-01,0.0,1,5.016563e-01,1,1
20231008,1.7,5.868241e-01,0.0,1,1.662144e-15,0,0
20240331,2.3,2.500000e-01,0.0,1,0.000000e+00,0,0
20240421,6.2,2.500000e-01,0.0,1,0.000000e+00,0,0
20240526,4.0,2.500000e-01,0.0,1,3.404109e-15,0,0
20240707,4.8,4.131759e-01,0.0,1,0.000000e+00,0,0
20240714,5.1,3.015369e-02,0.0,1,1.138065e-01,0,0
